In [1]:
# Importação as bibliotecas necessárias para conexão e coleta dos dados
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
# Variaveis ambientes

host = os.environ.get('HOST_POSTGRE')
user = os.environ.get('USER_POSTGRE')
password = os.environ.get('PASSWORD_POSTGRE')

In [3]:
# Descrevo os parâmetros de conexão conforme configurado
db_server_info = {'host': host, 
                  'porta': 5432, 
                  'dbname': 'dbs_vacina',
                  'user': user, 
                  'password': password} 

conn_str = '{}://{}:{}@{}:{}/{}'.format('postgresql+psycopg2', 
                                  db_server_info['user'], 
                                  db_server_info['password'], 
                                  db_server_info['host'], 
                                  db_server_info['porta'], 
                                  db_server_info['dbname']) 

alchemyEngine = create_engine(conn_str, pool_recycle=3600)

postgreSQLConnection = alchemyEngine.connect() 

In [4]:
# Encontrando o id relacionado ao Distrito Federal

pd.read_sql_query("SELECT * FROM public.tbl_cidade WHERE uf = 'DF'", postgreSQLConnection)

,id,nm_municipio,uf,lat,lon,cd_ibge_reduzido
0,5300108,Brasília,DF,-15.8049,-47.88683,530010


In [5]:
# Dataframe de CEPs do DF

dfCepsDF = pd.read_sql_query("SELECT * FROM public.tbl_cep WHERE cidade_id = 5300108", postgreSQLConnection)
dfCepsDF.head()

,id,cep,cidade_id,lat,lon
0,18098,70002,5300108,-47.879545,-15.790642
1,18099,70003,5300108,-47.880634,-15.790649
2,18100,70037,5300108,-47.882790,-15.805816
3,18101,70040,5300108,-47.884265,-15.791226
4,18102,70041,5300108,-47.881478,-15.788280


In [6]:
# Dataframe de Estabelecimentos de Saúde do DF

ceps = dfCepsDF.cep

dfEstab = pd.read_sql_query("SELECT * FROM public.tbl_estabelecimento", postgreSQLConnection)

dfEstabDF = dfEstab.loc[dfEstab['fk_cep'].isin(ceps)].copy()
dfEstabDF.head()

,id,nm_razao_social,nm_fantasia,fk_cep,id_natureza_juridica,id_tipo_estabelecimento,id_atividade_principal,lat,lon
295958,7399197,SAMU 192 DF,SAMU DF USB 12,70334,1023.0,NaN,NaN,NaN,NaN
296021,7399219,SAMU 192 DF,SAMU DF USB 16,70334,1023.0,NaN,NaN,NaN,NaN
297299,7399235,SAMU 192 DF,SAMU DF USB 17,70334,1023.0,NaN,NaN,NaN,NaN
299701,7399243,SAMU 192 DF,SAMU DF USB 18,70334,1023.0,NaN,NaN,NaN,NaN
300114,7399308,SAMU 192 DF,SAMU DF USB 20 PMR,70334,1023.0,NaN,NaN,NaN,NaN


In [7]:
# Quantidade de estabelecimentos de saúde no DF

dfEstabDF.shape

(8815, 9)

In [8]:
# Informações gerais do dataframe de estabelecimentos de saúde do DF

dfEstabDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8815 entries, 295958 to 312267
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       8815 non-null   int64  
 1   nm_razao_social          8815 non-null   object 
 2   nm_fantasia              8815 non-null   object 
 3   fk_cep                   8815 non-null   object 
 4   id_natureza_juridica     8815 non-null   float64
 5   id_tipo_estabelecimento  4596 non-null   float64
 6   id_atividade_principal   4596 non-null   float64
 7   lat                      0 non-null      float64
 8   lon                      0 non-null      float64
dtypes: float64(5), int64(1), object(3)
memory usage: 688.7+ KB


In [9]:
# Dropando colunas com muitos valores ausentes

dfEstabDF.drop(columns = ['id_tipo_estabelecimento','id_atividade_principal','lat','lon'],axis=1,inplace=True)
dfEstabDF.head()

,id,nm_razao_social,nm_fantasia,fk_cep,id_natureza_juridica
295958,7399197,SAMU 192 DF,SAMU DF USB 12,70334,1023.0
296021,7399219,SAMU 192 DF,SAMU DF USB 16,70334,1023.0
297299,7399235,SAMU 192 DF,SAMU DF USB 17,70334,1023.0
299701,7399243,SAMU 192 DF,SAMU DF USB 18,70334,1023.0
300114,7399308,SAMU 192 DF,SAMU DF USB 20 PMR,70334,1023.0


In [10]:
# Dataframe dos estabelecimentos de saúde que foram postos de vacinação no Brasil

dfEstabVacinacao = pd.read_sql_query("SELECT DISTINCT id_estabelecimento FROM public.tbl_vacinacao", postgreSQLConnection)
dfEstabVacinacao.rename(columns={"id_estabelecimento": "id"},inplace=True)
dfEstabVacinacao.head()

,id
0,2427265
1,2621673
2,2432528
3,2469197
4,7204159


In [11]:
# Dataframe dos postos de vacinação do DF

dfPostosDF = pd.merge(dfEstabDF,dfEstabVacinacao)
dfPostosDF.head()

,id,nm_razao_social,nm_fantasia,fk_cep,id_natureza_juridica
0,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,70160,1040.0
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200,1023.0
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203,1023.0
3,10510,FUNDACAO UNIVERSIDADE DE BRASILIA,HOSPITAL UNIVERSITARIO DE BRASILIA,70310,1139.0
4,6362729,COMANDO DA MARINHA,HOSPITAL NAVAL DE BRASILIA,70310,1015.0


In [12]:
# Quantidade de postos de vacinação no DF

dfPostosDF.shape

(116, 5)

In [13]:
# Adicionando info de latitude e longitude nos postos de vacinação no DF

dfCepsDF.drop(columns=['id','cidade_id'],inplace=True)
dfPostosDF.rename(columns={"fk_cep":"cep"},inplace=True)
dfPostosDF = pd.merge(dfPostosDF,dfCepsDF,on='cep')
dfPostosDF.head()

,id,nm_razao_social,nm_fantasia,cep,id_natureza_juridica,lat,lon
0,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,70160,1040.0,-47.861282,-15.800669
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200,1023.0,-47.891575,-15.822105
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203,1023.0,-47.897709,-15.823910
3,10510,FUNDACAO UNIVERSIDADE DE BRASILIA,HOSPITAL UNIVERSITARIO DE BRASILIA,70310,1139.0,-47.885490,-15.798626
4,6362729,COMANDO DA MARINHA,HOSPITAL NAVAL DE BRASILIA,70310,1015.0,-47.885490,-15.798626


In [14]:
# Adicionando informação de natureza jurídica dos postos de vacinação no DF

dfNatJur = pd.read_sql_query("SELECT * FROM public.tbl_natureza_juridica", postgreSQLConnection)
dfNatJur.rename(columns={"id":"id_natureza_juridica"},inplace=True)
dfPostosDF = pd.merge(dfPostosDF,dfNatJur,on='id_natureza_juridica')
dfPostosDF.head()

,id,nm_razao_social,nm_fantasia,cep,id_natureza_juridica,lat,lon,natureza_juridica
0,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,70160,1040.0,-47.861282,-15.800669,ORGAO PUBLICO DO PODER LEGISLATIVO FEDERAL
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,70200,1023.0,-47.891575,-15.822105,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,70203,1023.0,-47.897709,-15.823910,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
3,5117666,UNIDADE BASICA DE SAUDE N 5 ASA NORTE,UBS 5 GRANJA DO TORTO,70636,1023.0,-47.905893,-15.819210,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
4,10758,UNIDADE BASICA DE SAUDE N 2 CRUZEIRO,UBS 2 CRUZEIRO,70640,1023.0,-47.944787,-15.789660,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...


In [ ]:
#Visão geral dos postos do DF

dfPostosDF.info()

In [ ]:
# Analisando os dados de natureza jurídica
dfPostosDF.natureza_juridica.value_counts()

In [ ]:
# Lista de todos os pontos de vacinação do DF encontrados na base de dados

pd.set_option("max_rows", None)
dfPostosDF.nm_fantasia

In [ ]:
# Plotando os postos de vacinação do DF em um mapa
import folium as fl
import geopandas

m = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

dfPostosGeo = geopandas.GeoDataFrame(dfPostosDF, 
geometry = geopandas.points_from_xy(dfPostosDF.lon, dfPostosDF.lat))

for i in range(0,len(dfPostosGeo)):
  fl.Marker(
    [dfPostosGeo.loc[i,'lat'],dfPostosGeo.loc[i,'lon']],
    tooltip= dfPostosGeo.loc[i,'nm_fantasia'],
    icon=fl.Icon(color='blue')
    ).add_to(m)

m

In [ ]:
# Algumas UBS estão bem distantes da área do DF. Como são 5 casos, vale a pena olhar caso a caso.

dfPostosGeo.loc[(dfPostosGeo['nm_fantasia'] == "UBS 1 GAMA")]

In [ ]:
dfPostosGeo.loc[(dfPostosGeo['nm_fantasia'] == "UBS 2 GAMA")]

In [ ]:
dfPostosGeo.loc[(dfPostosGeo['nm_fantasia'] == "UBS 3 GAMA")]

In [ ]:
dfPostosGeo.loc[(dfPostosGeo['nm_fantasia'] == "UBS 1 ENGENHO VELHO FERCAL")]

In [ ]:
dfPostosGeo.loc[(dfPostosGeo['nm_fantasia'] == "UBS 9 ENGENHO DAS LAGES GAMA")]

In [ ]:
#Verificando os dados na tabela da Secretária de Saúde do DF e vendo as USBs, foi notado que os CEPS estão corretos, mas as coordenadas lat/long estão bem diferentes. 
# http://www.saude.df.gov.br/wp-conteudo/uploads/joomla/238bab226de40145ad9645523074e6ce.pdf
# Será feita alteração das coordenadas no dataframe dfPostosDF e no dataframe dfPostosGeo

indices = [60,59,63,62,83]
lat_alt = [-16.020679,-16.036695,-16.005841,-16.04388,-15.600348]
lon_alt = [-48.084848,-48.06098,-48.052746,-48.254283,-47.8715]

for i, la, lo in zip(indices,lat_alt, lon_alt):
  dfPostosDF.iloc[i,5] = la
  dfPostosGeo.iloc[i,5] = la
  dfPostosDF.iloc[i,6] = lo
  dfPostosGeo.iloc[i,6] = lo

In [ ]:
# Teste para ver se o loop anterior deu certo
dfPostosDF.iloc[60,]

In [ ]:
# Plotando novamente as UBs que são pontos de vacinação
from folium.plugins import MeasureControl

m1 = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

ubs = fl.FeatureGroup(name='UBS DF - Pontos de vacinação Covid-19')

for i in range(0,len(dfPostosGeo)):
  fl.Marker([dfPostosGeo.loc[i,'lat'],dfPostosGeo.loc[i,'lon']],
  tooltip= dfPostosGeo.loc[i,'nm_fantasia'],
  icon=fl.Icon(color='blue')
  ).add_to(ubs)

 
m1.add_child(ubs)
m1.add_child(MeasureControl())
m1.add_child(fl.LatLngPopup())
fl.LayerControl().add_to(m1)
m1

In [ ]:
# Análise da tbl de vacinação no Brasil

dfVacinacao = pd.read_sql_query('SELECT hash_paciente,estabelecimento_id,data_aplicacao,dose_declarada FROM pos.tbl_vacinacao', postgreSQLConnection)
dfVacinacao.head()

In [ ]:
# Dataframe de vacinação do DF

postos = dfPostosDF.id
dfVacinacao = dfVacinacao.loc[dfVacinacao['estabelecimento_id'].isin(postos)]
dfVacinacao.head()

In [ ]:
#Verificando o tamanho do dataframe de vacinação do Df
dfVacinacao.shape

In [ ]:
# Dataframe de pacientes no DF

dfPaciente = pd.read_sql_query('SELECT hash,id_cidade,id_cep,id_grupo FROM pos.tbl_paciente WHERE id_cidade = 5300108', postgreSQLConnection)
dfPaciente.head()

In [ ]:
# Merge das informações do cep da residência do paciente e do estabelecimento onde ele foi vacinado

dfVacinacao.rename(columns={"hash_paciente":"hash"},inplace=True)
dfMerged = pd.merge(dfVacinacao,dfPaciente,on='hash')
dfMerged.head()

In [ ]:
# Tamanho do Merge
dfMerged.shape

In [ ]:
# Checando as informações
dfMerged.info()

In [ ]:
# Verificando a quantidade de valores nulos de CEP
print(dfMerged.id_cep.isna().sum()/len(dfMerged))

In [ ]:
# Quase 30% dos vacinados do DF não possuem número do CEP da residência na base de dados. Não será possível fazer a análise da distância até o posto de vacinação desses pacientes. Iremos dropar do dataframe.

dfMerged = dfMerged.dropna()
dfMerged.shape

In [ ]:
# Retirada das pessoas que tomaram 2 doses para manter no dataset apenas pacientes unicos

dfMerged = dfMerged[dfMerged.dose_declarada == 1]
dfMerged.dose_declarada.value_counts()

In [ ]:
# Verificação do shape
dfMerged.shape

In [ ]:
# Agrupando os pacientes de acordo com o posto de vacinação e cep da residência
dfMerged_group = dfMerged[['estabelecimento_id','id_cep']].copy()
dfMerged_group = dfMerged_group.groupby(['estabelecimento_id','id_cep']).size().reset_index(name='contador')
dfMerged_group.head(10)

In [ ]:
# Valor máximo do contador (Pessoas com o mesmo cep que foram se vacinar no mesmo lugar)
dfMerged_group.contador.max()

In [ ]:
# Soma da coluna contador (deve ser igual o shape do dfMerged)
dfMerged_group.contador.sum()

In [ ]:
# Adição de latitude e longitude dos postos de vacinação
dfPostosDF.rename(columns={"id":"estabelecimento_id"},inplace=True)
dfMerged_group = pd.merge(dfMerged_group,dfPostosDF[['lat','lon','estabelecimento_id']],on='estabelecimento_id')
dfMerged_group.rename(columns={"lat":"lat_estab","lon":"lon_estab"},inplace=True)
dfMerged_group.head()

In [ ]:
# Adição de latitude e longitude das residências dos pacientes
dfMerged_group = pd.merge(dfMerged_group,dfCepsDF)
dfMerged_group.rename(columns={"lat":"lat_resid","lon":"lon_resid"},inplace=True)
dfMerged_group.head()

In [ ]:
# Checando informações
dfMerged_group.info()

In [ ]:
# Calculando a distancia entre as duas coordenadas

import haversine as hs
distancia = []
dfMerged_group['coor_estab'] = list(zip(dfMerged_group.lat_estab,dfMerged_group.lon_estab))
dfMerged_group['coor_resid'] = list(zip(dfMerged_group.lat_resid,dfMerged_group.lon_resid))

for index,row in dfMerged_group.iterrows():
    distancia.append(hs.haversine(row['coor_estab'],row['coor_resid']))

dfMerged_group['distancia'] = distancia
dfMerged_group.head()


In [ ]:
# Análise da variável distância

dfMerged_group.distancia.describe()

In [ ]:
# Histograma da variável distância

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
sns.histplot(data=dfMerged_group,x='distancia')

In [ ]:
# Boxplot da variável distância

sns.boxplot(x=dfMerged_group["distancia"])

In [ ]:
# Investigando os ceps que são outliers no cálculo da distância

dfMerged_group[dfMerged_group.distancia > 60].id_cep.value_counts()

In [ ]:
# Cada CEP foi verificado no Google Maps e as coordenadas foram diferentes das informações do nosso dataset

ceps_outliers = [72410,72430,71503,70630,72594,72457,72415,73150,72549,73330]
cep_correcao = dfCepsDF.loc[dfCepsDF['id_cep'].isin(ceps_outliers)]

ind1 = cep_correcao.index
lat_alt = [-15.85177,-15.72028,-16.01446,-16.01721,-16.00695,-16.00918,-16.05279,-15.99645,-15.60197,-15.60946]
lon_alt = [-47.94817,-47.88104,-48.05283,-48.08201,-48.05993,-48.06418,-47.98158,-47.99895,-47.87104,-47.87104]

for i1,la, lo in zip(ind1,lat_alt, lon_alt):
  dfCepsDF.iloc[i1,1] = la
  dfCepsDF.iloc[i1,2] = lo

In [ ]:
# Corrigindo todo o dataframe e calculando as novas distâncias

dfMerged_group.drop(['lat_resid','lon_resid','coor_estab','coor_resid','distancia'],axis='columns',inplace=True)

dfMerged_group = pd.merge(dfMerged_group,dfCepsDF)
dfMerged_group.rename(columns={"lat":"lat_resid","lon":"lon_resid"},inplace=True)

dfMerged_group['coor_estab'] = list(zip(dfMerged_group.lat_estab,dfMerged_group.lon_estab))
dfMerged_group['coor_resid'] = list(zip(dfMerged_group.lat_resid,dfMerged_group.lon_resid))

distancia.clear()
for index,row in dfMerged_group.iterrows():
    distancia.append(hs.haversine(row['coor_estab'],row['coor_resid']))

dfMerged_group['distancia'] = distancia
dfMerged_group.head()


In [ ]:
# Análise da variável distância após correção

dfMerged_group.distancia.describe()

In [ ]:
# Boxplot da variável distância após alteração

sns.boxplot(x=dfMerged_group["distancia"])

In [ ]:
# Distâncias de forma decrescente
dfMerged_group.sort_values(by='distancia',ascending=False,inplace=True)
dfMerged_group.head()

In [ ]:
# Lista das residências de pacientes que percorreram mais de 30km para ir até o posto de vacinação

resid_maior30 = dfMerged_group[dfMerged_group.distancia > 30].id_cep.value_counts()
df_resid_maior30 = dfCepsDF.loc[dfCepsDF['id_cep'].isin(resid_maior30.index)]
df_resid_maior30.reset_index(inplace=True)
df_resid_maior30.head()

In [ ]:
# Análise dos pontos de vacinação mais cheios

dfUBS_count = dfMerged_group.groupby('estabelecimento_id')['contador'].agg('sum')
dfUBS_count = pd.merge(dfUBS_count,dfPostosDF[['estabelecimento_id','nm_fantasia','lat','lon']],on='estabelecimento_id')
dfUBS_count.sort_values(ascending=False,by='contador',inplace=True)
top10UBS = dfUBS_count.head(10)
top10UBS.reset_index(inplace=True,drop=True)
top10UBS


In [ ]:
# Criando marcadores

from folium.plugins import MeasureControl

m1 = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

ubs = fl.FeatureGroup(name='UBS DF - Pontos de vacinação Covid-19')
for i in range(0,len(dfPostosGeo)): 
  fl.Marker([dfPostosGeo.loc[i,'lat'],dfPostosGeo.loc[i,'lon']],
  icon=fl.Icon(color='blue'),
  tooltip= dfPostosGeo.loc[i,'nm_fantasia'],
  ).add_to(ubs)
 
resid = fl.FeatureGroup(name='Deslocamento > 30 km')
for i in range(0,len(df_resid_maior30)):
  fl.Circle([df_resid_maior30.loc[i,'lat'],df_resid_maior30.loc[i,'lon']],
  color='red',
  ).add_to(resid)

top10 = fl.FeatureGroup(name='Pontos de vacinação mais cheios')
for i in range(0,len(top10UBS)):
  fl.Marker([top10UBS.loc[i,'lat'],top10UBS.loc[i,'lon']],
  icon=fl.Icon(color='red'),
  tooltip= top10UBS.loc[i,'nm_fantasia'],
  ).add_to(top10)

m1.add_child(resid)
m1.add_child(ubs)
m1.add_child(top10)
m1.add_child(MeasureControl())
m1.add_child(fl.LatLngPopup())
fl.LayerControl().add_to(m1)

m1.save('mapa.html')

m1

In [ ]:
#Construção do mapa de calor

dfHeat = dfMerged.groupby(['data_aplicacao','id_cep']).size().reset_index(name='contador')
dfHeat = pd.merge(dfHeat,dfCepsDF[['lat','lon','id_cep']],on='id_cep')
plotHeat = dfHeat[['lat','lon','contador']].values.tolist()

m2 = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

from folium.plugins import HeatMap
heatmap = HeatMap(plotHeat,radius=18).add_to(m2)
heatmap.save('mapa_calor.html')
m2



In [ ]:
# Fechando conexão
postgreSQLConnection.close()